In [3]:
import pycolmap
import torch 
from pathlib import Path
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from src.geometry.point_transformation import *
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
requires_grad = True

In [ ]:
scene_folder = Path('../data/south-building')
images_folder = scene_folder / 'images'
database_path = scene_folder / 'database.db'
output_path = scene_folder / 'undistorted_images'

reconstruction = pycolmap.Reconstruction(output_path / 'sparse')

img_id = 1
img = reconstruction.images[img_id]
cam_id = 1

extrinsic_matrix = get_extrinsic_params(img.cam_from_world)
f, cx, cy = reconstruction.cameras[cam_id].params
width, height = reconstruction.cameras[cam_id].width, reconstruction.cameras[cam_id].height
znear, zfar = 1, 5
intrinsic_matrix = get_intrinsic_opengl_params(f, f, height, width, zfar=zfar, znear=znear)

colors = np.array([p.color for p in reconstruction.points3D.values()]) / 255.0
homogeneous_points = convert_to_homogenous(get_points_xyz(reconstruction))
camera_coordinates = homogeneous_points @ extrinsic_matrix.T
clip_coordinates = camera_coordinates @ intrinsic_matrix.T
point_ids = cull_coordinates_ids(clip_coordinates, camera_coordinates, zfar=zfar, znear=znear)

ndc_coordinates = to_ndc_coordinates(clip_coordinates[point_ids])
c = colors[point_ids]

screen_coordinates = to_screen_coordinates(ndc_coordinates, width, height, zfar, znear)
image = mpimg.imread(images_folder / f'{img.name}')
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.scatter(screen_coordinates[:, 0], -screen_coordinates[:, 1], s=2, c=c)
plt.subplot(1, 2, 2)
plt.imshow(image)

In [1]:
print(f"# filtered points {sum(point_ids)}")
print(f"# total points {reconstruction.num_points3D()}")

NameError: name 'point_ids' is not defined

In [21]:
pcd = o3d.io.read_point_cloud(str(scene_folder / 'sparse.ply'))

Initialize covariance matrices

In [28]:
pcd.estimate_covariances(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=3))
print(np.asarray(pcd.covariances).shape)

(57713, 3, 3)


In [34]:
cov_3d = torch.tensor(np.asarray(pcd.covariances), requires_grad=requires_grad, device=device)
cov_3d

tensor([[[ 4.4004e-06,  5.1835e-06, -2.3480e-06],
         [ 5.1835e-06,  2.3757e-05, -9.6698e-07],
         [-2.3480e-06, -9.6698e-07,  1.4362e-06]],

        [[ 4.7358e-06, -4.8224e-06, -2.5215e-06],
         [-4.8224e-06,  1.5954e-04,  3.8990e-05],
         [-2.5215e-06,  3.8990e-05,  9.9217e-06]],

        [[ 1.4094e-05, -1.5892e-05, -3.1668e-06],
         [-1.5892e-05,  1.7975e-05,  3.4742e-06],
         [-3.1668e-06,  3.4742e-06,  8.8423e-07]],

        ...,

        [[ 8.3156e-07,  4.9832e-06,  2.4029e-06],
         [ 4.9832e-06,  3.0314e-05,  1.6523e-05],
         [ 2.4029e-06,  1.6523e-05,  1.6917e-05]],

        [[ 8.1634e-06,  3.2351e-06,  6.2868e-06],
         [ 3.2351e-06,  2.8877e-06,  3.4254e-06],
         [ 6.2868e-06,  3.4254e-06,  5.3849e-06]],

        [[ 7.9381e-07, -2.9239e-06,  3.7494e-06],
         [-2.9239e-06,  1.0953e-05, -1.5756e-05],
         [ 3.7494e-06, -1.5756e-05,  3.8413e-05]]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)

In [25]:
o3d.visualization.draw_geometries([pcd])